In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
colors = sns.color_palette("hls")

import random
import datetime
import numpy as np
import gc
import os
import pickle

import scipy.sparse as sp
import pandas as pd
from tqdm.auto import tqdm

from itertools import islice, cycle
from more_itertools import pairwise
from implicit.nearest_neighbours import TFIDFRecommender
from scipy.sparse.linalg import svds

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import seaborn as sns
sns.set(style = 'whitegrid')
sns.set(rc = {'figure.figsize':(17, 9)})

from IPython import display
display.display(display.HTML('<style>.container { width:80% !important; }</style>'))
display.display(display.HTML('<style>.prompt { min-width:10ex !important; }</style>'))
display.display(display.HTML('<style>div#notebook { font-size:12px !important; }</style>'))

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = DeprecationWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)
warnings.simplefilter(action = 'ignore', category = RuntimeWarning)
warnings.filterwarnings("ignore", message = "numpy.dtype size changed")
warnings.filterwarnings("ignore", message = "numpy.ufunc size changed")
pd.options.mode.chained_assignment = None
pd.set_option('mode.chained_assignment', None)


path = '../input/'

DEBUG = False

user_id = 'user_id'
item_id = 'item_id'
rank = 'emoji'
date = 'date'
top_N = 10

In [2]:
data = pd.read_parquet(path + "player_starts_train.parquet")
data.head(3)

,date,user_id,item_id,watch_time,is_autorized
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0


In [3]:
videos = pd.read_parquet(path + "videos.parquet")
videos.head(3)

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,None,0,В видео я обывательским взглядом расскажу про ...,Технологии и интернет,2022-12-08 13:53:05+03:00,391382,0,0,NaN,0.0,0.0,NaN
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",None,0,Видео с канала УЦ Академия безопасности (ab-dp...,Образование,2022-03-24 09:19:15+03:00,125922,26,0,NaN,0.0,0.0,0.0
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,None,0,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Авто-мото,2022-03-19 17:41:49+03:00,436570,2,0,NaN,NaN,0.0,0.0


In [4]:
submission = pd.read_csv(path + "sample_submission.csv")
submission.head(3)

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"['video_0', 'video_0', 'video_0', 'video_0', '..."
2,user_29734049,"['video_0', 'video_0', 'video_0', 'video_0', '..."


In [5]:
def reduce_mem_usage(df, verbose = True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

reduce_mem_usage(data);
reduce_mem_usage(videos);
reduce_mem_usage(submission);

Mem. usage decreased to 1934.70 Mb (27.5% reduction)
Mem. usage decreased to 181.48 Mb (31.7% reduction)
Mem. usage decreased to  1.48 Mb (0.0% reduction)


In [6]:
%%time

user_ids = data['user_id'].unique()
print(len(user_ids))
print(data.shape, videos.shape)

if DEBUG:
    user_ids = user_ids[:288007]
    
    data = data[data[user_id].isin(user_ids)]
    item_ids = data[item_id].unique()    
    videos = videos[videos[item_id].isin(item_ids)]
    gc.collect()

#data.drop_duplicates(subset = ['user_id', 'item_id'], keep = False)    
    
print(len(user_ids))
print(data.shape, videos.shape)

28800762
(69954380, 5) (2320660, 15)
28800762
(69954380, 5) (2320660, 15)
CPU times: user 23.4 s, sys: 362 ms, total: 23.8 s
Wall time: 23.9 s


In [7]:
data[date] = pd.to_datetime(data[date], format = '%Y-%m-%d')

In [8]:
remove = [
    'video_description', 'author_title', 'tv_title', 'season', 'publicated', 'channel_sub', 
    'tv_sub', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08'
]

videos.drop(remove, axis = 1, inplace = True)

In [9]:
data = data.merge(videos, how = 'left', on = 'item_id')
del videos; gc.collect()

0

In [10]:
# Исключаем видео без watch_time

data = data[data['watch_time']!=-1]
gc.collect()

18

# Для проверки моделей возмем часть тренировочных данных, которые не будут участвовать в тренировке моделей.

In [11]:
# Отделим 5 дней на вилидацию, остальное на тренировку.

time_split = '2023-08-16'

train = data[data[date] < time_split]
valid = data[data[date] >= time_split]
gc.collect()

del data

print('количество пользователей в тренировочных данных ', len(train[user_id].unique()))
print('количество пользователей в валидационных данных ', len(valid[user_id].unique()))

print(train.shape, valid.shape)

количество пользователей в тренировочных данных  22661012
количество пользователей в валидационных данных  6205665
(53526149, 8) (12850741, 8)


In [12]:
train.head(3)

,date,user_id,item_id,watch_time,is_autorized,video_title,category_title,duration
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0,"ШОК! ТОП 10 ФЕЙЛОВ, КОТОРЫЕ ВАС ТОЧНО УДИВЯТ!",Развлечения,213718
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0,песня-В ЛУННОМ СИЯНИИ,Музыка,129562
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0,Юлия Пересильд о дружбе с мужчинами,Лайфстайл,242006


In [13]:
valid.head(3)

,date,user_id,item_id,watch_time,is_autorized,video_title,category_title,duration
56242112,2023-08-16 21:50:43+03:00,user_8599328,video_1009706,0,0,Марина Александрова раскрыла секреты воспитани...,Развлечения,25151
56242113,2023-08-16 21:25:20+03:00,user_13657319,video_1858819,2,0,ЧВК «Вагнер». Контракт с Родиной,Фильмы,1572096
56242114,2023-08-16 08:56:56+03:00,user_1229611,video_965653,12,0,"Шоу Воли, 1 выпуск",Телепередачи,3542000


# Метрика

In [14]:
def apk(actual, predicted, k = 10):
    if not actual:
        return 0.0

    if len(predicted) > k:
        predicted = predicted[:k]
        
    actual = list(set(actual))
    predicted = list(set(predicted))

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k = 10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [15]:
# Проверим метрику

actual    = ['video_0', 'video_1', 'video_2',  'video_3',  'video_5', 'video_6',  'video_7', 'video_9', 'video_9']
predicted = ['video_0', 'video_0', 'video_11', 'video_13', 'video_5', 'video_16', 'video_7', 'video_9', 'video_10']

mapk(actual, predicted, k = 10)

0.909750566893424

In [16]:
valid_sub = valid.groupby(user_id)[item_id].apply(lambda x: list(x.values)).reset_index()
valid_sub.head(5)

,user_id,item_id
0,user_1000,[video_1125407]
1,user_100000,[video_1411578]
2,user_1000000,[video_1104709]
3,user_10000000,"[video_352866, video_435990, video_352866, vid..."
4,user_10000001,[video_1250322]


# 1. Расчет рекомендаций из топ-10 самых просматриваемых.

In [17]:
class PopularRecommender():
    def __init__(self, max_K = 10, days = 30, item_column = item_id, dt_column = date):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days = self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users = None, N = 10):
        recs = self.recommendations[:N]      
        
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [18]:
pop_model = PopularRecommender(days = 10, dt_column = date)
pop_model.fit(train)

In [19]:
top10_recs = pop_model.recommend()
list(top10_recs)

['video_1594159',
 'video_170129',
 'video_1508131',
 'video_68646',
 'video_1761620',
 'video_302657',
 'video_2052690',
 'video_803844',
 'video_1508623',
 'video_255770']

In [20]:
with open("top10_recs.pkl", "wb") as file:
    pickle.dump(top10_recs, file)

In [21]:
# Проверяем из каких категорий получился топ-10

for item in top10_recs:
    print(train[['category_title', 'video_title']][train[item_id] == item].values[0])

['Телепередачи' 'Выжить в Дубае, 7 выпуск']
['Искусство' 'Филипп Янковский высказался об Андрее Тарковском']
['Лайфстайл'
 'Анастасия Попова («Бэби-тур») высказалась об обнаженных сценах']
['Телепередачи' 'Выжить в Дубае, 8 выпуск']
['Телепередачи' 'Соловьёв LIVE | Круглосуточный канал']
['Телепередачи' 'Выжить в Дубае, 6 выпуск']
['Телепередачи' 'Прямой эфир ТНТ']
['Телепередачи' 'Прямой эфир Звезда']
['Телепередачи' 'Выжить в Дубае, 1 выпуск']
['Телепередачи' 'Прямой эфир ТВ-3']


In [22]:
pred_sub = valid_sub.copy()
pred_sub = pred_sub.groupby(user_id)[item_id].apply(lambda x: list(top10_recs)).reset_index()
pred_sub.head(5)

,user_id,item_id
0,user_1000,"[video_1594159, video_170129, video_1508131, v..."
1,user_100000,"[video_1594159, video_170129, video_1508131, v..."
2,user_1000000,"[video_1594159, video_170129, video_1508131, v..."
3,user_10000000,"[video_1594159, video_170129, video_1508131, v..."
4,user_10000001,"[video_1594159, video_170129, video_1508131, v..."


In [23]:
# Считаем метрику

mapk(valid_sub[item_id].values, pred_sub[item_id].values, k = 10)

0.020709042050586744

# Проверим как изменяется метрика от количесва анализуруемых дней

In [24]:
top10_recs = []

for day in tqdm(range(1, 15, 2)):
    pop_model = PopularRecommender(days = day, dt_column = date)
    pop_model.fit(train)
    top10 = pop_model.recommend()
    top10 = list(set(top10))
    
    pred_sub = valid_sub.copy()
    pred_sub = pred_sub.groupby(user_id)[item_id].apply(lambda x: list(top10)).reset_index()
    m = mapk(valid_sub[item_id].values, pred_sub[item_id].values, k = 10)
    
    print('кол-во дней для анализа', day, 'метрика', np.round(m, 5))
    top10_recs.append(top10)

  0%|          | 0/7 [00:00<?, ?it/s]

кол-во дней для анализа 1 метрика 0.01846
кол-во дней для анализа 3 метрика 0.02071
кол-во дней для анализа 5 метрика 0.02071
кол-во дней для анализа 7 метрика 0.02003
кол-во дней для анализа 9 метрика 0.02003
кол-во дней для анализа 11 метрика 0.02003
кол-во дней для анализа 13 метрика 0.01917


In [25]:
temp = pd.concat([train, valid], ignore_index = True)

In [26]:
pop_model = PopularRecommender(days = 5, dt_column = date)
pop_model.fit(temp)
top10 = pop_model.recommend()
top10_recs.append(top10)

del temp

In [27]:
with open("top10_recs_full.pkl", "wb") as file:
    pickle.dump(top10_recs, file)

# Выводы: 
## Для "холодного старта" - метрика хорошая
## Но результат отрицательный, практически все рекомендовано из одной категории.

# 2. Расчет рекомендаций из топ 10 самых просматриваемых в разных категориях.

In [28]:
# Топ самых популярных категорий

top = train['category_title'].value_counts()
print('всего категорий', len(top))
top_25 = top[:25]
top_25

всего категорий 46


Телепередачи             13754869
Видеоигры                 4300145
Сериалы                   4155944
Развлечения               3695543
Лайфстайл                 3257533
Животные                  2591015
Юмор                      2271842
Разное                    1946020
Спорт                     1559732
Фильмы                    1533569
Кулинария                 1444345
Музыка                    1439069
Авто-мото                 1239389
Детям                     1000688
Путешествия                909293
Образование                860023
Искусство                  672293
Аниме                      659491
Здоровье                   597886
Лайфхаки                   486997
Интервью                   486381
Хобби                      471990
Сад и огород               441820
Технологии и интернет      430942
Эзотерика                  429106
Name: category_title, dtype: int64

In [29]:
top10_recs2 = []

for video_title in tqdm(top_25[:10].index): 
    temp = train[train['category_title'] == video_title] # берем лучшее из топ-10 категорий    
    pop_model = PopularRecommender(days = 10, dt_column = date)
    pop_model.fit(temp)
    top_recs = pop_model.recommend()
    top10_recs2.append(top_recs[:1][0])
    
top10_recs2 = list(set(top10_recs2))
top10_recs2

  0%|          | 0/10 [00:00<?, ?it/s]

['video_1508131',
 'video_1594159',
 'video_1486704',
 'video_291331',
 'video_1425649',
 'video_1135323',
 'video_1836195',
 'video_2159643',
 'video_1867381',
 'video_1623184']

In [30]:
with open("top10_recs2.pkl", "wb") as file:
    pickle.dump(top10_recs2, file)

In [31]:
# Проверяем из каких категорий получился топ-10

for item in top10_recs2:
    print(train[['category_title', 'video_title']][train[item_id] == item].values[0])

['Лайфстайл'
 'Анастасия Попова («Бэби-тур») высказалась об обнаженных сценах']
['Телепередачи' 'Выжить в Дубае, 7 выпуск']
['Юмор' 'Смешные животные ? / №1']
['Разное' 'Из-за чего на самом деле сорвалась свадьба Елены Воробей']
['Сериалы'
 'Зимородок (2022) 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 серия']
['Фильмы' '❗️ "ЗА ОТЦА" - фильм про месть']
['Видеоигры'
 'NFS Unbound: первые впечатления! Глоток свежатины в аркадах! ?']
['Спорт'
 'Хейбати VS Самброс. ЗАРУБА. Назир. Ушу-Мастер VS Мири. Ялымов VS Намитов за пояс. Алисафа. НОКАУТ']
['Развлечения' 'Давай сделаем ЭТО на кухне | Найти любовь в Интернете']
['Животные' 'Симпатичный #корелла Кеша огрызается ☺️']


In [32]:
# Создадим файл решения

pred_sub = valid_sub.copy()
pred_sub = pred_sub.groupby(user_id)[item_id].apply(lambda x: list(top10_recs2)).reset_index()
pred_sub.head(5)

,user_id,item_id
0,user_1000,"[video_1508131, video_1594159, video_1486704, ..."
1,user_100000,"[video_1508131, video_1594159, video_1486704, ..."
2,user_1000000,"[video_1508131, video_1594159, video_1486704, ..."
3,user_10000000,"[video_1508131, video_1594159, video_1486704, ..."
4,user_10000001,"[video_1508131, video_1594159, video_1486704, ..."


In [33]:
# Считаем метрику

mapk(valid_sub[item_id].values, pred_sub[item_id].values, k = 10)

0.012748343333556887

# 3. Удалим отрицательные эмоции.

In [34]:
emotions = pd.read_csv(path + 'emotions.csv')
del emotions['date']
emotions.head(3)

,user_id,item_id,type,emoji
0,user_21883648,video_2247834,v_top,v_top
1,user_24016046,video_2247834,v_top,v_top
2,user_20748867,video_22669,v_top,v_top


In [35]:
train = train.merge(emotions, how = 'left', on = ['user_id', 'item_id'])
#del emotions; gc.collect()

In [36]:
train.columns

Index(['date', 'user_id', 'item_id', 'watch_time', 'is_autorized',
       'video_title', 'category_title', 'duration', 'type', 'emoji'],
      dtype='object')

In [37]:
train['type_emoji'] = train['type'] + '_' + train['emoji']
train['type_emoji'].value_counts()

v_top_v_top                        234034
pos_emotions_Like                  114048
pos_emotions_New_Fire               39093
pos_emotions_Heart                  33794
pos_emotions_Happy_glasses          15085
pos_emotions_Laugh                  12402
pos_emotions_Happy_Star_glasses      7394
neg_emotions_Dislike                 1341
neg_emotions_Sickness                 288
neg_emotions_Gape                     209
neg_emotions_Angry                    190
neg_emotions_Crying                   159
neg_emotions_Closed_mauth              52
Name: type_emoji, dtype: int64

In [38]:
%%time

train2 = train.copy()
train2 = train2[train2['type_emoji'] != 'neg_emotions_Closed_mauth']
train2 = train2[train2['type_emoji'] != 'neg_emotions_Crying']
train2 = train2[train2['type_emoji'] != 'neg_emotions_Gape']
train2 = train2[train2['type_emoji'] != 'neg_emotions_Angry']
#train2 = train2[train2['type_emoji'] != 'neg_emotions_Sickness']
#train2 = train2[train2['type_emoji'] != 'neg_emotions_Dislike']
gc.collect();

CPU times: user 28.1 s, sys: 26 s, total: 54.1 s
Wall time: 2min 12s


In [39]:
pop_model = PopularRecommender(days = 10, dt_column = date)
pop_model.fit(train2)

In [40]:
top10_recs3 = pop_model.recommend()
list(top10_recs3)

['video_1594159',
 'video_170129',
 'video_1508131',
 'video_68646',
 'video_1761620',
 'video_302657',
 'video_2052690',
 'video_803844',
 'video_1508623',
 'video_255770']

In [41]:
with open("top10_recs3.pkl", "wb") as file:
    pickle.dump(top10_recs3, file)

In [42]:
# Создадим файл решения

pred_sub = valid_sub.copy()
pred_sub = pred_sub.groupby(user_id)[item_id].apply(lambda x: list(top10_recs3)).reset_index()
pred_sub.head(5)

,user_id,item_id
0,user_1000,"[video_1594159, video_170129, video_1508131, v..."
1,user_100000,"[video_1594159, video_170129, video_1508131, v..."
2,user_1000000,"[video_1594159, video_170129, video_1508131, v..."
3,user_10000000,"[video_1594159, video_170129, video_1508131, v..."
4,user_10000001,"[video_1594159, video_170129, video_1508131, v..."


In [43]:
# Считаем метрику

mapk(valid_sub[item_id].values, pred_sub[item_id].values, k = 10)

0.020709042050586744

In [55]:
# Проверяем из каких категорий получился топ-10

for item in top10_recs3:
    print(train[['category_title', 'video_title']][train[item_id] == item].values[0])

['Телепередачи' 'Выжить в Дубае, 7 выпуск']
['Искусство' 'Филипп Янковский высказался об Андрее Тарковском']
['Лайфстайл'
 'Анастасия Попова («Бэби-тур») высказалась об обнаженных сценах']
['Телепередачи' 'Выжить в Дубае, 8 выпуск']
['Телепередачи' 'Соловьёв LIVE | Круглосуточный канал']
['Телепередачи' 'Выжить в Дубае, 6 выпуск']
['Телепередачи' 'Прямой эфир ТНТ']
['Телепередачи' 'Прямой эфир Звезда']
['Телепередачи' 'Выжить в Дубае, 1 выпуск']
['Телепередачи' 'Прямой эфир ТВ-3']


# 4. Удалим записи с продолжительностью просмотра 0.1% и менее.

In [44]:
print(train2.shape)

train2['views percentage'] = train2['watch_time'] / train2['duration'] * 100
train2 = train2[train2['views percentage'] == train2['views percentage']]
train2 = train2[train2['views percentage'] >= 0.1]

del train2['views percentage']
print(train2.shape)

(53657182, 11)
(3797385, 11)


In [45]:
pop_model = PopularRecommender(days = 10, dt_column = date)
pop_model.fit(train2)

In [46]:
top10_recs4 = pop_model.recommend()
list(top10_recs4)

['video_1761620',
 'video_2052690',
 'video_803844',
 'video_255770',
 'video_1628773',
 'video_1979002',
 'video_1738263',
 'video_505893',
 'video_956001',
 'video_892296']

In [47]:
# Создадим файл решения

pred_sub = valid_sub.copy()
pred_sub = pred_sub.groupby(user_id)[item_id].apply(lambda x: list(top10_recs4)).reset_index()
pred_sub.head(5)

,user_id,item_id
0,user_1000,"[video_1761620, video_2052690, video_803844, v..."
1,user_100000,"[video_1761620, video_2052690, video_803844, v..."
2,user_1000000,"[video_1761620, video_2052690, video_803844, v..."
3,user_10000000,"[video_1761620, video_2052690, video_803844, v..."
4,user_10000001,"[video_1761620, video_2052690, video_803844, v..."


In [48]:
with open("top10_recs4.pkl", "wb") as file:
    pickle.dump(top10_recs4, file)

In [49]:
# Считаем метрику

mapk(valid_sub[item_id].values, pred_sub[item_id].values, k = 10)

0.012670803371237646

In [56]:
# Проверяем из каких категорий получился топ-10

for item in top10_recs4:
    print(train[['category_title', 'video_title']][train[item_id] == item].values[0])

['Телепередачи' 'Соловьёв LIVE | Круглосуточный канал']
['Телепередачи' 'Прямой эфир ТНТ']
['Телепередачи' 'Прямой эфир Звезда']
['Телепередачи' 'Прямой эфир ТВ-3']
['Телепередачи' 'Прямой эфир Пятница']
['Телепередачи' 'Первый канал. Прямой эфир']
['Телепередачи' 'Прямой эфир Ю-ТВ']
['Телепередачи' 'Прямой эфир НТВ']
['Телепередачи' 'Прямой эфир ТНТ4']
['Телепередачи' 'Прямой эфир МУЗ-ТВ']


# 5. Дополнительно предлагается добавить еще несколько топ-10, которые рекомендавать случайным образом.

In [50]:
import random

top10_recs_random = []

for i in tqdm(range(5)):
    top10_recs_temp = []
    for video_title in top_25[0:10].sample(10).index:
        temp = train2[train2['category_title'] == video_title]  # выбираем категорию    
        pop_model = PopularRecommender(days = 10, dt_column = date)
        pop_model.fit(temp)
        top_recs = pop_model.recommend()
    
        s = random.choice([1, 2, 3]) # случайный выбор из топ-3 каждой категории
    
        if video_title != 'Новости':
            top10_recs_temp.append(top_recs[s-1:s][0])
        else:
            top10_recs_temp.append(top_recs[:1][0])
        
    top10_recs_random.append(top10_recs_temp)
    
top10_recs_random

  0%|          | 0/5 [00:00<?, ?it/s]

[['video_769766',
  'video_1480251',
  'video_2002914',
  'video_1508131',
  'video_1667523',
  'video_2331189',
  'video_1761620',
  'video_1025464',
  'video_1155457',
  'video_897700'],
 ['video_458888',
  'video_1921490',
  'video_1600266',
  'video_2002914',
  'video_1286888',
  'video_291331',
  'video_1810173',
  'video_2247834',
  'video_1873054',
  'video_2052690'],
 ['video_2002914',
  'video_1810173',
  'video_1286888',
  'video_754241',
  'video_1025464',
  'video_1873054',
  'video_2247834',
  'video_881410',
  'video_986887',
  'video_1761620'],
 ['video_1025464',
  'video_1155457',
  'video_769766',
  'video_754241',
  'video_1480251',
  'video_1759381',
  'video_2331189',
  'video_1921490',
  'video_1761620',
  'video_1667523'],
 ['video_2002914',
  'video_2286239',
  'video_458888',
  'video_8186',
  'video_897700',
  'video_1667523',
  'video_803844',
  'video_1480251',
  'video_769766',
  'video_754241']]

In [51]:
with open("top10_recs_random.pkl", "wb") as file:
    pickle.dump(top10_recs_random, file)

In [52]:
%%time

import random

with open('top10_recs_random.pkl', 'rb') as f:
    top10_recs_global = pickle.load(f)

#pred_sub = pd.read_csv(path + "sample_submission.csv")

pred_sub = valid[[user_id]].copy()
pred_sub['recs'] = '[]'

pred_sub['recs'] = pred_sub['recs'].apply(lambda x: random.choice(top10_recs_global))
    
pred_sub

CPU times: user 3.13 s, sys: 277 ms, total: 3.4 s
Wall time: 3.97 s


,user_id,recs
56242112,user_8599328,"[video_458888, video_1921490, video_1600266, v..."
56242113,user_13657319,"[video_458888, video_1921490, video_1600266, v..."
56242114,user_1229611,"[video_769766, video_1480251, video_2002914, v..."
56242115,user_11310214,"[video_769766, video_1480251, video_2002914, v..."
56242116,user_2365988,"[video_1025464, video_1155457, video_769766, v..."
...,...,...
69954378,user_15478739,"[video_769766, video_1480251, video_2002914, v..."
69954379,user_25783543,"[video_458888, video_1921490, video_1600266, v..."
69954380,user_3507470,"[video_458888, video_1921490, video_1600266, v..."
69954381,user_13128840,"[video_458888, video_1921490, video_1600266, v..."


In [53]:
# Считаем метрику

mapk(valid_sub[item_id], pred_sub['recs'], k = 10)

0.004482076457711597

# Вывод:

# При добавлении вариативности (выбор топ из разных категорий "2. Расчет рекомендаций из топ 10 самых просматриваемых в разных категориях"), метрика качества снижается, однако в допустимых пределах. 
## Поэтому следует применять именно этот метод - так мы больше охватим интресов новых пользователей.